In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

In [3]:
account = pd.read_csv('data/account.csv')
card_test = pd.read_csv('data/card_test.csv')
card_train = pd.read_csv('data/card_train.csv')
client = pd.read_csv('data/client.csv')
disp = pd.read_csv('data/disp.csv')
district = pd.read_csv('data/district.csv')
loan_test = pd.read_csv('data/loan_test.csv')
loan_train = pd.read_csv('data/loan_train.csv')
trans_test = pd.read_csv('data/trans_test.csv')
trans_train = pd.read_csv('data/trans_train.csv')


In [5]:
client

,"client_id;""birth_number"";""district_id"""
0,"1;""706213"";18"
1,"2;""450204"";1"
2,"3;""406009"";1"
3,"4;""561201"";5"
4,"5;""605703"";5"
...,...
5364,"13955;""456030"";1"
5365,"13956;""430406"";1"
5366,"13968;""680413"";61"
5367,"13971;""626019"";67"


706163